## Some comparisons of filtering and smoothing (can be with or without the viewer)

### Instruction reminders for your tasks 3) and 4)
**You must implement your filtering / smoothing in the FilterSmoother class in the file _Filters.py_. This means also that you can use the visualisation through the Localizer, but you can call filter / smooth also directly from here, without the Localizer. Note also, that you must make use of the given models and the robot simulator in RobotSim! Inspect _Localizer.py_ for inspiration on how to do that!**

### Hint:
You can of course implement the different evaluations (task 4) in a regular .py-file and run Python from the command line! If you use the notebook, you need to restart the kernel after every change in a .py-file that is called from the notebook.


In [5]:
# In this cell, you can write your own "main" to run and evaluate your 
# implementation without using the visualisation (should be considerably faster!)

from models import *
from Filters import FilterSmoother

import numpy as np
import matplotlib.pyplot as plt # for plotting the average errors in comparison
import random
import view_control.Localizer as Localizer
# ...  

In [13]:
steps = 100
r, c = 8,8 

sm = StateModel(r,c)
loc = Localizer(sm,0,5)
tot_err = 0

for step in range(steps):
    ret, tsX, tsY, tsH, srX, srY, eX, eY, error, fPositions = loc.update()
    # print(error)
    # print(srX,tsX)
    # print(fPositions)
    tot_err += error
avg_err = tot_err/steps
print(avg_err)


1.35
